In [12]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
import pickle
import os

/home/el10/Downloads/yes/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/el10/Downloads/yes/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/el10/Downloads/yes/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/el10/Downloads/yes/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/el10/Downloads/yes/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/el10/Downloads/yes/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478

In [13]:
model_name = "mlp"
train = pd.read_csv('nyc_demand_train.csv', parse_dates=['timeStamp'])
X = train.drop(['demand', 'timeStamp'], axis=1)

In [14]:
len_train = int(len(X) * 0.75)
len_valid = len(X) - len_train
test_fold = [-1]*len_train + [0]*len_valid
ps = PredefinedSplit(test_fold)

In [15]:
regr = MLPRegressor(solver='lbfgs', verbose=True)

In [16]:
hidden_layer_size = [(5,), (10,), (15,), (20,), (25,), (30,), (35,), (40,), (10,10), (20,20), (30,30), (40,40), (24,7,52), (24,7,12), (24,6,4), (24,12)]

In [17]:
param_grid = {'hidden_layer_sizes': hidden_layer_size,
             'alpha': [0.01, 0.1, 1.0, 10.0]}
regr_cv = GridSearchCV(estimator=regr,
            param_grid=param_grid,
            cv=ps,
            scoring='r2',
            verbose=2,
            n_jobs=-1)

In [18]:
regr_pipe = Pipeline([('regr_cv', regr_cv)])
regr_pipe.fit(X, y=train['demand'])

Fitting 1 folds for each of 64 candidates, totalling 64 fits
[CV] alpha=0.01, hidden_layer_sizes=(5,) .............................
[CV] alpha=0.01, hidden_layer_sizes=(10,) ............................
[CV] alpha=0.01, hidden_layer_sizes=(15,) ............................
[CV] alpha=0.01, hidden_layer_sizes=(20,) ............................
[CV] .............. alpha=0.01, hidden_layer_sizes=(5,), total=   1.6s
[CV] alpha=0.01, hidden_layer_sizes=(25,) ............................
[CV] ............. alpha=0.01, hidden_layer_sizes=(10,), total=   1.7s
[CV] alpha=0.01, hidden_layer_sizes=(30,) ............................
[CV] ............. alpha=0.01, hidden_layer_sizes=(15,), total=   2.4s
[CV] alpha=0.01, hidden_layer_sizes=(35,) ............................
[CV] ............. alpha=0.01, hidden_layer_sizes=(20,), total=   2.6s
[CV] alpha=0.01, hidden_layer_sizes=(40,) ............................
[CV] ............. alpha=0.01, hidden_layer_sizes=(25,), total=   3.0s
[CV] alpha=0.01,

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   37.3s


[CV] .............. alpha=1.0, hidden_layer_sizes=(15,), total=   2.3s
[CV] alpha=1.0, hidden_layer_sizes=(30,) .............................
[CV] ........... alpha=0.1, hidden_layer_sizes=(24, 12), total=   4.3s
[CV] alpha=1.0, hidden_layer_sizes=(35,) .............................
[CV] .............. alpha=1.0, hidden_layer_sizes=(20,), total=   2.7s
[CV] alpha=1.0, hidden_layer_sizes=(40,) .............................
[CV] .............. alpha=1.0, hidden_layer_sizes=(25,), total=   3.0s
[CV] alpha=1.0, hidden_layer_sizes=(10, 10) ..........................
[CV] .............. alpha=1.0, hidden_layer_sizes=(35,), total=   4.6s
[CV] alpha=1.0, hidden_layer_sizes=(20, 20) ..........................
[CV] .............. alpha=1.0, hidden_layer_sizes=(30,), total=   4.8s
[CV] alpha=1.0, hidden_layer_sizes=(30, 30) ..........................
[CV] .............. alpha=1.0, hidden_layer_sizes=(40,), total=   7.0s
[CV] alpha=1.0, hidden_layer_sizes=(40, 40) ..........................
[CV] .

[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:  1.3min finished


Pipeline(memory=None,
     steps=[('regr_cv', GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
       error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1...   pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=2))])

In [19]:
with open(model_name + '.pkl', 'wb') as f:
    pickle.dump(regr_pipe, f)


In [20]:
cv_results = pd.DataFrame(regr_pipe.named_steps['regr_cv'].cv_results_)
cv_results.sort_values(by='rank_test_score', inplace=True)
cv_results.head()

/home/el10/Downloads/yes/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/el10/Downloads/yes/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/el10/Downloads/yes/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('std_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,param_hidden_layer_sizes,params,rank_test_score,split0_test_score,split0_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
11,8.273699,0.005423,0.962259,0.972261,0.01,"(40, 40)","{'alpha': 0.01, 'hidden_layer_sizes': (40, 40)}",1,0.962259,0.972261,0.0,0.0,1.110223e-16,0.0
60,8.080205,0.005749,0.961891,0.970964,10,"(24, 7, 52)","{'alpha': 10.0, 'hidden_layer_sizes': (24, 7, ...",2,0.961891,0.970964,0.0,0.0,1.110223e-16,0.0
47,3.613286,0.003597,0.960891,0.968180,1,"(24, 12)","{'alpha': 1.0, 'hidden_layer_sizes': (24, 12)}",3,0.960891,0.968180,0.0,0.0,0.000000e+00,0.0
26,7.722956,0.010485,0.960602,0.969702,0.1,"(30, 30)","{'alpha': 0.1, 'hidden_layer_sizes': (30, 30)}",4,0.960602,0.969702,0.0,0.0,0.000000e+00,0.0
13,5.541118,0.001738,0.960521,0.969376,0.01,"(24, 7, 12)","{'alpha': 0.01, 'hidden_layer_sizes': (24, 7, ...",5,0.960521,0.969376,0.0,0.0,0.000000e+00,0.0
